# 도구변수(IV)
- 작성자: 고려대학교 경제학과 한치록 교수, 데이터사이언스팀 이창훈 과장

간편한 사용을 위하여 `bok_da` 라이브러리에 도구변수 추정 모듈을 제공한다. 

* `ivregress`: 2SLS와 LIML

이 문서에서는 Stata와 결과를 비교하고자 한다. Python에서 Stata의 실행에 관한 자세한 내용은 `7-01`과 `7-02` 매뉴얼을 참고하라. 이 문서들에서는 `bok_da` 라이브러리에 구현된 간편 모듈을 사용한다.

[3_06]: 06%20BOK%20Library%20I%20(LS).ipynb
[3_07]: 07%20BOK%20Library%20II%20(IV).ipynb
[3_08]: 08%20BOK%20Library%20III%20(Time%20Series).ipynb

#### **(주의) 본 매뉴얼에서 Stata 기능은 라이선스 이슈로 아직까지는 BIDAS 환경에서 사용할 수 없다. 매뉴얼에서 stata 관련 코드는 주석처리하였다. 로컬환경(내부망, 인터넷망)에서 활용하는 경우 주석해제 하여 사용할 수 있다.**

## ivregress

도구변수 추정을 위한 모듈은 `ivregress`이다. `method` 인자는 "2sls", "liml" 중 하나로서 각자 fit된 model (`IVResults`)을 리턴한다. 이 결과들은 `linearmodels`을 monkey patch한 것임에 유의하라. 결과를 보여 줄 때에는 `summary()` 메쏘드가 아니라 `summary` 프로퍼티를 사용한다.

### 2SLS

#### 추정

먼저 2SLS (two-stage least squares)를 살펴보자. 디폴트 표준오차는 'unadjusted'이다.

In [1]:
import bok_da as bd
import pandas as pd
from bok_da.linear.lm import ivregress

In [2]:
df = pd.read_stata('../data/crime4.dta')
fm = (
    'lcrmrte~[lprbarr+lpolpc~ltaxpc+lmix]+'
    'lprbconv+lprbpris+lavgsen+ldensity+'
    'lwcon+lwtuc+lwtrd+lwfir+lwser+lwmfg+lwfed+lwsta+lwloc+'
    'lpctymle+lpctmin+'
    'west+central+urban+C(year)'
)
tsls = ivregress(fm, df)
print(tsls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                lcrmrte   R-squared:                      0.8044
Estimator:                    IV-2SLS   Adj. R-squared:                 0.7960
No. Observations:                 630   F-statistic:                    2166.1
Date:                Thu, May 22 2025   P-value (F-stat)                0.0000
Time:                        15:29:34   Distribution:                 chi2(26)
Cov. Estimator:            unadjusted                                         
                                                                              
                                Parameter Estimates                                
                 Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------
Intercept          -2.5969     1.1776    -2.2053     0.0274     -4.9050     -0.2889
lprbconv           -0.3981     0

이분산에 견고한 표준오차는 다음과 같이 구한다.

In [3]:
tsls = ivregress(fm, df, vce='r')
print(tsls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                lcrmrte   R-squared:                      0.8044
Estimator:                    IV-2SLS   Adj. R-squared:                 0.7960
No. Observations:                 630   F-statistic:                    3454.6
Date:                Thu, May 22 2025   P-value (F-stat)                0.0000
Time:                        15:29:39   Distribution:                 chi2(26)
Cov. Estimator:                robust                                         
                                                                              
                                Parameter Estimates                                
                 Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------
Intercept          -2.5969     1.1443    -2.2695     0.0232     -4.8397     -0.3542
lprbconv           -0.3981     0

클러스터 표준오차는 다음과 같이 구한다.

In [4]:
tsls = ivregress(fm, df, vce='cl', cluster='county')
print(tsls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                lcrmrte   R-squared:                      0.8044
Estimator:                    IV-2SLS   Adj. R-squared:                 0.7960
No. Observations:                 630   F-statistic:                    1051.9
Date:                Thu, May 22 2025   P-value (F-stat)                0.0000
Time:                        15:29:41   Distribution:                 chi2(26)
Cov. Estimator:             clustered                                         
                                                                              
                                Parameter Estimates                                
                 Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------
Intercept          -2.5969     2.1219    -1.2239     0.2210     -6.7558      1.5619
lprbconv           -0.3981     0

위에서 `cluster='county'` 대신에 `cluster=df.county` 또는 `cluster=df['county']`라고 해도 좋다.

Stata 결과와 비교해 보면 똑같은 것을 알 수 있다.

In [5]:
from bok_da.stata import Stata

In [4]:
# df = pd.read_stata('crime4.dta')
# stata = Stata('/Applications/Stata', 'mp')
# stata.get_ready()
# stata.use(df, force = True)
# stata.run(
#     'ivregress 2sls lcrmrte (lprbarr lpolpc = ltaxpc lmix) '
#     'lprbconv lprbpris lavgsen ldensity lwcon-lwloc '
#     'lpctymle lpctmin west central urban i.year, '
#     'vce(cl county)'
# )

. ivregress 2sls lcrmrte (lprbarr lpolpc = ltaxpc lmix) lprbconv lprbpris lavgs
> en ldensity lwcon-lwloc lpctymle lpctmin west central urban i.year, vce(cl co
> unty)

Instrumental variables 2SLS regression            Number of obs   =        630
                                                  Wald chi2(26)   =    1051.91
                                                  Prob > chi2     =     0.0000
                                                  R-squared       =     0.8044
                                                  Root MSE        =     .25314

                                (Std. err. adjusted for 90 clusters in county)
------------------------------------------------------------------------------
             |               Robust
     lcrmrte | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
     lprbarr |  -.3785279   .2142882    -1.77   0.077    -.7985252    .0414693
    

#### First-stage relevance tests

In [7]:
df = pd.read_stata('../data/crime4.dta')
fm = (
    'lcrmrte~[lprbarr+lpolpc~ltaxpc+lmix]+'
    'lprbconv+lprbpris+lavgsen+ldensity+'
    'lwcon+lwtuc+lwtrd+lwfir+lwser+lwmfg+lwfed+lwsta+lwloc+'
    'lpctymle+lpctmin+'
    'west+central+urban+C(year)'
)
tsls = ivregress(fm, df)
print(tsls.first_stage)

          First Stage Estimation Results          
                               lprbarr      lpolpc
--------------------------------------------------
R-squared                       0.4455      0.3239
Partial R-squared               0.2107      0.1227
Shea's R-squared                0.1346      0.0784
Partial F-statistic             84.105      44.063
P-value (Partial F-stat)      1.11e-16    1.11e-16
Partial F-stat Distn          F(2,603)    F(2,603)
========================== =========== ===========
Intercept                      -1.9352     -11.538
                             (-1.6402)   (-6.9933)
lprbconv                       -0.1736      0.3275
                             (-7.3666)    (9.9395)
lprbpris                       -0.0592      0.0137
                             (-0.9853)    (0.1625)
lavgsen                        -0.0066     -0.0264
                             (-0.1332)   (-0.3829)
ldensity                       -0.2151     -0.0047
                             (-

#### Regressor Endogeneity Test

In [8]:
tsls.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 9.6667
P-value: 0.0080
Distributed: chi2(2)
WaldTestStatistic, id: 0x2824c8e5dc0

In [9]:
tsls.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 4.6109
P-value: 0.0103
Distributed: F(2,601)
WaldTestStatistic, id: 0x2824ab1ff50

#### Overidentification Test

내생적 설명변수의 개수와 추가 도구변수의 개수가 같으므로 도구변수 외생성 검정(과다식별검정)은 가능하지 않다.

In [10]:
tsls.wooldridge_overid

Invalid test statistic
Test requires more instruments than endogenous variables.
Wooldridge's score test of overidentification
InvalidTestStatistic, id: 0x2824c8e46e0

### Limited Information Maximum Likelihood

`LIML(Limited Information Maximum Likelihood)`은 단일 방정식만 보고 최대우도 원리로 내생변수를 보정하는 IV 추정법으로, 특히 도구변수가 약할 때(weak instruments) 2SLS보다 효율성과 편향 보정 측면에서 유리하다. 도구변수가 내생변수를 충분히 설명하지 못해 1단계 회귀에서 설명력이 약한 상황에서 LIML이 편향을 일부 조정해 줄 수 있다. LIML은 `method="liml"` 인자를 주어 구할 수 있다. 위 예에서 LIML 추정을 하고 클러스터 표준오차를 사용할 경우 다음과 같이 한다.

In [11]:
liml = ivregress(fm, df, method='liml', vce='cl', cluster='county')
print(liml.summary)

                          IV-LIML Estimation Summary                          
Dep. Variable:                lcrmrte   R-squared:                      0.8044
Estimator:                    IV-LIML   Adj. R-squared:                 0.7960
No. Observations:                 630   F-statistic:                    1051.9
Date:                Thu, May 22 2025   P-value (F-stat)                0.0000
Time:                        15:30:10   Distribution:                 chi2(26)
Cov. Estimator:             clustered                                         
                                                                              
                                Parameter Estimates                                
                 Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------
Intercept          -2.5969     2.1219    -1.2239     0.2210     -6.7558      1.5619
lprbconv           -0.3981     0

Stata 결과와 비교하면 동일함을 알 수 있다.

In [10]:
# stata = Stata('/Applications/Stata', 'mp')
# stata.get_ready()
# stata.run(
#     "use crime4, clear\n"
#     'ivregress liml lcrmrte (lprbarr lpolpc = ltaxpc lmix) '
#     'lprbconv lprbpris lavgsen ldensity lwcon-lwloc '
#     'lpctymle lpctmin west central urban i.year, '
#     'vce(cl county)'
# )


. use crime4, clear

. ivregress liml lcrmrte (lprbarr lpolpc = ltaxpc lmix) lprbconv lprbpris lavgs
> en ldensity lwcon-lwloc lpctymle lpctmin west central urban i.year, vce(cl co
> unty)

Instrumental variables LIML regression            Number of obs   =        630
                                                  Wald chi2(26)   =    1051.91
                                                  Prob > chi2     =     0.0000
                                                  R-squared       =     0.8044
                                                  Root MSE        =     .25314

                                (Std. err. adjusted for 90 clusters in county)
------------------------------------------------------------------------------
             |               Robust
     lcrmrte | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
     lprbarr |  -.3785279   .2142882    -1.77   0.077    -.79